# Environment Setup



## Cloning the MinGPT git into the Collab Directory


In [2]:
!git clone https://github.com/karpathy/minGPT

Cloning into 'minGPT'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175
Receiving objects: 100% (175/175), 1.37 MiB | 1.69 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [3]:
%cd /content/minGPT

/content/minGPT


## Connect the Google Drive


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Training Data 


## Download and Parse the Basic-Bot Training Data

### Upload and Data Parsing


In [5]:
import re

#Load the Data
with open('/content/gdrive/My Drive/U9APJ3XKN.txt') as f:
    BasicBot_text = f.readlines()

#Parse the Data
BasicBot_text = [ x for x in BasicBot_text if "has joined the channel" not in x] #Remove channel joins
BasicBot_text = [ x for x in BasicBot_text if "has left the channel" not in x] #Remove channel exits
BasicBot_text = [ re.sub("<https.*?>", "<HTTPS>", x) for x in BasicBot_text] #Replace any http links with <HTTPS>
#BasicBot_text = [re.sub("\n$", "<|endoftext|>", x) for x in BasicBot_text] #Signify the end of each message <|endoftext|>
#BasicBot_text = ["<|startoftext|>" + x for x in BasicBot_text] #Signify the start of each message (<|startoftext|>)
BasicBot_text_size=len("".join(BasicBot_text))

## Download and Parse a Reddit Conversation Corpus
Lets add a little variety to the Basic-Bot using some random comments from Reddit conversations. I've noticed that with Kenny's training data alone, the model either overfits or is a random combination of messages strung together. Lets some percentage of extra text from random reddit messages to make the output more interesting. If we want to give the bot different "personalities" we can restrict to specific subreddits.




In [6]:
pip install convokit

     |████████████████████████████████| 143kB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 21.2MB/s 
     |████████████████████████████████| 245kB 47.8MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
  Created wheel for convokit: filename=convokit-2.4.2-cp36-none-any.whl size=163344 sha256=9fc27a5bf2965b0f4d07cde70f8b6601ac09393e7c43b53beb95ef5042b70a70
  Stored in directory: /root/.cache/pip/wheels/27/05/c7/f8abe991947da6c5a60c1994e41e889b823f4ea388750c9cac
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=4197f2a481aba2e3aec477d79830bae048c232c35bf5d797ca5e30a4c0fc5608
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=c1636e475be740030903bc90f4a11a9fafe0eb17663ffde60d5601ef1afd83bd
  Stored in directory: /root/.cache/pip/wheels/ba/

In [7]:
from convokit import Corpus, download
corpus = Corpus(filename=download("reddit-corpus-small"))

In [8]:
inject_prop=0.2
char_count=0
Reddit_content=[]
while char_count < inject_prop*BasicBot_text_size: 
  newtext=corpus.random_utterance().text +"\n"
  char_count = char_count + len(newtext)
  Reddit_content.append(newtext)

In [9]:
Reddit_content=[x for x in Reddit_content if "[deleted]" not in x] #Remove comments flagged as deleted
Reddit_content=[x for x in Reddit_content if "[removed]" not in x] #Remove comments flagged as removed
Reddit_content=[re.sub(r'https?:\/\/\S*', '<HTTPS>', x, flags=re.MULTILINE) for x in Reddit_content] #Replace urls with a generic code

## Combine Corpus


In [10]:
import random

train_dat = BasicBot_text + Reddit_content
random.shuffle(train_dat)
train_dat = "".join(train_dat)
len(train_dat)

255237

# minGPT

## Preprocess the Training Data


In [11]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [12]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [13]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [14]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]

        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [15]:
block_size = 128 
train_dataset = CharDataset(train_dat, block_size)

data has 255237 characters, 119 unique.


In [28]:
#Save the train_dataset 
import pickle
training_save_name="training_dataset.pickle"
path = F"/content/gdrive/My Drive/{training_save_name}"
with open(path, 'wb') as f:
    pickle.dump(train_dataset, f)

## Construct the GPT Model

In [31]:
#Load the training data 
training_save_name="training_dataset.pickle"
path = F"/content/gdrive/My Drive/{training_save_name}"
with open(path, 'rb') as f:
    train_dataset = pickle.load(f)

In [32]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

10/20/2020 03:17:45 - INFO - mingpt.model -   number of parameters: 2.540749e+07


## Train the Model

In [37]:
#Make room for the model training
import gc
gc.collect()

2166

In [38]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=1, batch_size=128, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

Streaming output truncated to the last 5000 lines.
epoch 1 iter 744: train loss 1.41451. lr 5.498209e-04:  37%|███▋      | 744/1994 [12:50<21:32,  1.03s/it]

epoch 1 iter 744: train loss 1.41451. lr 5.498209e-04:  37%|███▋      | 745/1994 [12:50<21:42,  1.04s/it]

epoch 1 iter 745: train loss 1.42305. lr 5.496899e-04:  37%|███▋      | 745/1994 [12:51<21:42,  1.04s/it]

epoch 1 iter 745: train loss 1.42305. lr 5.496899e-04:  37%|███▋      | 746/1994 [12:51<21:35,  1.04s/it]

epoch 1 iter 746: train loss 1.40630. lr 5.495587e-04:  37%|███▋      | 746/1994 [12:52<21:35,  1.04s/it]

epoch 1 iter 746: train loss 1.40630. lr 5.495587e-04:  37%|███▋      | 747/1994 [12:52<21:28,  1.03s/it]

epoch 1 iter 747: train loss 1.42367. lr 5.494274e-04:  37%|███▋      | 747/1994 [12:53<21:28,  1.03s/it]

epoch 1 iter 747: train loss 1.42367. lr 5.494274e-04:  38%|███▊      | 748/1994 [12:53<21:32,  1.04s/it]

epoch 1 iter 748: train loss 1.41968. lr 5.492959e-04:  38%|███▊      | 748/1994 [12:54<21:32

In [39]:
#Save the Model
model_save_name = 'Basic-Bot_trained.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

## Test the Model

In [32]:
model_save_name = 'Basic-Bot_trained.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
path

In [45]:
from mingpt.utils import sample

context = "wtf "
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 200, temperature=1.5, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

wtf only and 36 pm
Bilbo/Frodor a coit](10 amp; Twith scrift cards :expressionless:
near new?
if you can even get an Uber out there
As opposed to an app to be bast fals facilex being card.
which no one jo
